In [6]:
import pandas as pd
import numpy as np
import time

#shape[0] : number of rows
#shape[1] : number of columns
def sigmoid(X, weight):
    z = np.dot(X, weight)
   # print("Z dim =", z.shape[0])
   # print("z =", z)
    return 1 / (1 + np.exp(-z))

#print(1/(1+np.exp(-0.17)))

#print(1+np.exp(1))
#print(1 + np.exp([1]))
#print(1 + np.exp([1,2]))
print(np.dot([[1,2,3],[4,5,6]],[1,2,3]))

def gradient_descent(X, h, y):
#X.T transpose of X
#y.shape[0] is sample size (m in the learning material), we divide by to find avg (batch mode)
    return np.dot(X.T, (h - y)) / y.shape[0]

def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

data = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
print("rows = {}".format(data.shape[0]))
print("columns = {}".format(data.shape[1]))
pd.DataFrame(data.dtypes).rename(columns = {0:"dtype"})

df = data.copy()

df['class'] = df['Churn'].apply(lambda x : 1 if x == "Yes" else 0)
#df.shape[1]
X = df[['tenure','MonthlyCharges']].copy()
y = df['class'].copy()

intercept = np.ones((X.shape[0], 1))
X = np.concatenate((intercept, X), axis=1)
#print(X.shape[0],X.shape[1])

#alpha: learning rate
alpha=0.001

start_time = time.time()

num_iter = 20000
# inputs
theta = np.zeros(X.shape[1])
for i in range(num_iter):
    #print("X=", X)
    #print("X.T=", X.T)
    #print("Theta=", theta)
    h = sigmoid(X, theta)
    #print("h=", h)
    #print("y=", y)
    gradient = gradient_descent(X, h, y)
    #print("gradient =", gradient)
    theta = update_weight_loss(theta, alpha, gradient)

print("Training time (Log Reg using Gradient descent):" + str(time.time() - start_time) + " seconds")
print("Learning rate: {}\nIteration: {}".format(alpha, num_iter))

result = sigmoid(X, theta)

f = pd.DataFrame(np.around(result, decimals=6)).join(y)
f['pred'] = f[0].apply(lambda x : 0 if x < 0.5 else 1)
print("Accuracy (Loss minimization):")
print(f.loc[f['pred']==f['class']].shape[0] / f.shape[0] * 100)

#For Confusion Matrix
YActual = f['class'].tolist()
YPredicted =  f['pred'].tolist()

#print(YActual)
#print(YPredicted)

TP = 0
TN = 0
FP = 0
FN = 0

for l1,l2 in zip(YActual, YPredicted):
    if (l1 == 1 and  l2 == 1):
        TP = TP + 1
    elif (l1 == 0 and l2 == 0):
        TN = TN + 1
    elif (l1 == 1 and l2 == 0):
        FN = FN + 1
    elif (l1 == 0 and l2 == 1):
        FP = FP + 1

print("Confusion Matrix: ")

print("TP=", TP)
print("TN=", TN)
print("FP=", FP)
print("FN=", FN)

# Precision = TruePositives / (TruePositives + FalsePositives)
# Recall = TruePositives / (TruePositives + FalseNegatives)


P = TP/(TP + FP)
R = TP/(TP + FN)

print("Precision = ", P)
print("Recall = ", R)

#F-Measure = (2 * Precision * Recall) / (Precision + Recall), sometimes called F1

F1 = (2* P * R)/(P + R)

print("F score = ")
print(F1)


[14 32]
rows = 7043
columns = 21
Training time (Log Reg using Gradient descent):3.366811752319336 seconds
Learning rate: 0.001
Iteration: 20000
Accuracy (Loss minimization):
78.1769132471958
Confusion Matrix: 
TP= 937
TN= 4569
FP= 605
FN= 932
Precision =  0.6076523994811932
Recall =  0.5013376136971642
F score = 
0.5493990032248608


In [ ]:
train_size = int(df.shape[0] * 80 / 100)
train = df[:train_size]
test = df[train_size:]

In [45]:
import numpy as np
import pandas as pd
import time

# ... [your functions: sigmoid, gradient_descent, update_weight_loss]

X_train = train[['tenure','MonthlyCharges']].copy()
y_train = train['class'].copy()

X_test = test[['tenure','MonthlyCharges']].copy()
y_test = test['class'].copy()

# Adding intercept for train data
intercept_train = np.ones((X_train.shape[0], 1))
X_train = np.concatenate((intercept_train, X_train), axis=1)

# Adding intercept for test data
intercept_test = np.ones((X_test.shape[0], 1))
X_test = np.concatenate((intercept_test, X_test), axis=1)

alpha = 0.001
start_time = time.time()

num_iter = 20000
theta = np.zeros(X_train.shape[1])

for i in range(num_iter):
    h = sigmoid(X_train, theta)
    gradient = gradient_descent(X_train, h, y_train)
    theta = update_weight_loss(theta, alpha, gradient)

print("Training time (Log Reg using Gradient descent):" + str(time.time() - start_time) + " seconds")
print("Learning rate: {}\nIteration: {}".format(alpha, num_iter))

# Validation on test data
result_test = sigmoid(X_test, theta)
f_test = pd.DataFrame(np.around(result_test, decimals=6)).join(y_test.reset_index(drop=True))
f_test['pred'] = f_test[0].apply(lambda x : 0 if x < 0.5 else 1)
print("Test Accuracy:")
print(f_test.loc[f_test['pred'] == f_test['class']].shape[0] / f_test.shape[0] * 100)

# Confusion Matrix on test data
YActual_test = f_test['class'].tolist()
YPredicted_test = f_test['pred'].tolist()

TP = sum((a == 1) & (b == 1) for a, b in zip(YActual_test, YPredicted_test))
TN = sum((a == 0) & (b == 0) for a, b in zip(YActual_test, YPredicted_test))
FP = sum((a == 0) & (b == 1) for a, b in zip(YActual_test, YPredicted_test))
FN = sum((a == 1) & (b == 0) for a, b in zip(YActual_test, YPredicted_test))

print("Test Confusion Matrix: ")
print("TP=", TP)
print("TN=", TN)
print("FP=", FP)
print("FN=", FN)

P = TP / (TP + FP)
R = TP / (TP + FN)

print("Test Precision = ", P)
print("Test Recall = ", R)

F1 = (2 * P * R) / (P + R)
print("Test F score = ")
print(F1)

Training time (Log Reg using Gradient descent):4.404621124267578 seconds
Learning rate: 0.001
Iteration: 20000
Test Accuracy:
79.13413768630234
Test Confusion Matrix: 
TP= 193
TN= 922
FP= 106
FN= 188
Test Precision =  0.6454849498327759
Test Recall =  0.5065616797900262
Test F score = 
0.5676470588235294


In [44]:
theta

array([-0.66292322, -0.0554298 ,  0.01855074])

In [43]:
X_test.head()

,tenure,MonthlyCharges
5634,1,19.90
5635,22,54.70
5636,14,46.35
5637,64,90.25
5638,1,19.95


In [22]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,class
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1


In [46]:
import seaborn as sns

sns.plot(gradient)

AttributeError: module 'seaborn' has no attribute 'plot'

In [23]:
df.shape[0]

7043

In [31]:
train.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,class
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1


In [25]:
train.shape

(5634, 22)

In [26]:
test.shape

(1409, 22)

In [30]:
f.head()

,0,class,pred
0,0.459116,0,0
1,0.186839,0,0
2,0.557393,1,1
3,0.087081,0,0
4,0.633243,1,1
